In [1]:
import pandas as pd
import numpy as np
import json
import os

In [2]:
os.listdir()

['.DS_Store',
 'conversations_v1',
 '.ipynb_checkpoints',
 'Dialogues-1.ipynb',
 'Dialogues.ipynb',
 'pilot-1.csv']

In [3]:
pilot = pd.read_csv("pilot-1.csv")
#pilot.head(5)
# Batch_4018758_batch_results,,,,,,,,,,,,,,,,,,,,,,,,,, # First line

In [4]:
path = "conversations_v1/conversations_v1"
files = os.listdir(path)
print("Reading From Directory ", path)
condition_dict = dict()
dialogue_stats = {"success_true":0, "success_false":0, "finished_true":0, "finished_false":0}
nvalids, ntextrows = 0, 0
diagTurn = []
storeDf = pd.DataFrame(columns = ["num", "fileName", "id", "duration" ,"success", "finished", "type", "conversation"])
for i,fn in enumerate(files, 1):
    print()
    print("* * *  {}. {}  * * *".format(i,fn))
    try:
        fp = os.path.join(path, fn)
        df = pd.read_json(fp)
        condition = df.condition.dropna().iloc[0]
        condition_dict[condition] = condition_dict.get(condition, 0) + 1
        df = df.iloc[1:]
        worker_id = df["text"].iloc[0]
        print("Worker_id: {}".format(worker_id), end="\t")
        time0 = df["timestamp"].iloc[0]
        time1 = df[df["finished"]==1.0]["timestamp"].iloc[0]
        seconds = (time1 - time0).seconds
        finished = df["finished"].max()
        success = df["success"].max()
        ntextrows += df.text.replace("", np.nan).dropna().count()
        nvalids += 1
        dialogue_stats["finished_" + ["false", "true"][int(finished)]] = round(ntextrows / nvalids, 2)
        dialogue_stats["success_" + ["false", "true"][int(success)]] = round(ntextrows / nvalids, 2)
    except Exception as err:
        print(err)
        print("Missing text or worker ID.")
        # print(df) # uncomment to see the dataframe
    else:
        print("Duration: {} \tFinished: {}\tSuccess: {}".format(seconds, finished, success))
        print("\nConversation\n------------")
        series = df[["name", "text"]].set_index("name").squeeze()
        convo = ''
#         count = 0
        for k, v in zip(series.index, series.values):
            print(" {}: {}".format(k, v))
            convo += " {}: {}\n".format(k, v)
        storeDf = storeDf.append({"num": i, "fileName":fn , "id":worker_id, "duration":seconds, "success":success, "finished":finished, "type":condition, "conversation":convo}, ignore_index=True)
#         if success == 1.0 : 
#             diagTurn.append()
print(" * * * END OF PROCESSING * * *")
print("\nMean Worktime: {:1.2f}sec".format(pilot.WorkTimeInSeconds.mean()))
print("\nNumber of files:", len(files))
print("\nValid Dialogues:", nvalids)
print("\nCondition Counter: {}".format(condition_dict))
print("\nDialogue Stats: {}".format(dialogue_stats))

Reading From Directory  conversations_v1/conversations_v1

* * *  1. b6d828156a6f4f7e922ff4d708a8084b.json  * * *
Worker_id: A142ZRU284W9O	Duration: 15 	Finished: 1.0	Success: 0.0

Conversation
------------
 S: A142ZRU284W9O
 S: Pink is the target.
 L: Do you mean magenta ?
 S: No.
 L: 
 S: Hot pink.
 S: Select hot pink, the one on the left.

* * *  2. 4fb0e0cb9b7e4330aa8f01c204c611a8.json  * * *
Worker_id: A35BY30TC8WCL4	Duration: 25 	Finished: 1.0	Success: 0.0

Conversation
------------
 S: A35BY30TC8WCL4
 S: It's green, kind of.
 L: I could not understand you.
 S: It's a green color.
 L: 
 S: It's kind of turquoise.

* * *  3. a4b917629f264955901f1bd7abde0988.json  * * *
'text'
Missing text or worker ID.

* * *  4. 4534f31223654992b295e86bba6b41fe.json  * * *
Worker_id: A22KRF782ELLB0	Duration: 34 	Finished: 1.0	Success: 0.0

Conversation
------------
 S: A22KRF782ELLB0
 S: pale green
 L: 
 S: light green
 S: green

* * *  5. 051bb1abb9dc4518879590ccb7fbd826.json  * * *
Worker_id: A

* * *  51. 6c1c6e95db3647c18fa0e5b7d4bd5cd4.json  * * *
Worker_id: ASWZO1WY1UHEP	Duration: 17 	Finished: 1.0	Success: 0.0

Conversation
------------
 S: ASWZO1WY1UHEP
 S: Violate
 L: I could not understand you.
 S: Green
 L: 

* * *  52. 62ad043656384035bef3cd54e4fd42ee.json  * * *
'text'
Missing text or worker ID.

* * *  53. 3e9ee70b3c684858bd907e596a20ca9a.json  * * *
Worker_id: A2H8GNSEVUJXYL	Duration: 41 	Finished: 1.0	Success: 1.0

Conversation
------------
 S: A2H8GNSEVUJXYL
 S: 
 L: I could not understand you.
 S: what
 L: I could not understand you.
 S: so
 L: I could not understand you.
 S: what is my task
 L: I could not understand you.
 S: target is left
 L: I could not understand you.
 S: target is blue
 L: 
 S: now
 S: what
 S: target is red

* * *  54. 6921753699ea4e309f3a2dac6d330d67.json  * * *
Worker_id: A3T61LJ1WNOL6Z	Duration: 22 	Finished: 1.0	Success: 1.0

Conversation
------------
 S: A3T61LJ1WNOL6Z
 S: ok
 L: I could not understand you.
 S: start
 L: I could not

In [5]:
print(" A Valid Structure Example:", fp)

 A Valid Structure Example: conversations_v1/conversations_v1/ec81742632b841299c232580f2ad7170.json


In [6]:
storeDf.head()

,num,fileName,id,duration,success,finished,type,conversation
0,1,b6d828156a6f4f7e922ff4d708a8084b.json,A142ZRU284W9O,15,0.0,1.0,split,S: A142ZRU284W9O\n S: Pink is the target.\n L...
1,2,4fb0e0cb9b7e4330aa8f01c204c611a8.json,A35BY30TC8WCL4,25,0.0,1.0,split,"S: A35BY30TC8WCL4\n S: It's green, kind of.\n..."
2,4,4534f31223654992b295e86bba6b41fe.json,A22KRF782ELLB0,34,0.0,1.0,far,S: A22KRF782ELLB0\n S: pale green\n L: \n S: ...
3,5,051bb1abb9dc4518879590ccb7fbd826.json,A9EVF0SO2IUJ0,10,0.0,1.0,far,S: A9EVF0SO2IUJ0\n S: Blue\n L: \n S: Green\n
4,7,f1baeaa7d46b482d97761d1eb338030e.json,A2MKXI4KCRRI7Y,67,0.0,1.0,close,S: A2MKXI4KCRRI7Y\n S: lightest one\n L: The ...


In [7]:
import ipywidgets
import seaborn as sns

In [8]:
@ipywidgets.interact
def show_dialog(success = [0.0, 1.0, 'none'], finished = [0.0, 1.0, 'none'], type=['split','far', 'close', 'none']):
    bool1 = storeDf["success"] == (success if success != 'none' else storeDf["success"])
    bool2 = storeDf["finished"] == (finished if finished != 'none' else storeDf["finished"])
    bool3 = storeDf["type"] == (type if type != 'none' else storeDf["type"])
    for index, row in storeDf[bool1 & bool2 & bool3].iterrows():
        print("* * *  {}. {}  * * *".format(row.num,row.fileName))
        print("Worker_id: {}".format(row.id), end="\t")
        print("Duration: {} \tFinished: {}\tSuccess: {}".format(row.duration, row.finished, row.success))
        print("\nConversation\n------------")
        print(row.conversation)
    return storeDf[bool1 & bool2 & bool3]
    

interactive(children=(Dropdown(description='success', options=(0.0, 1.0, 'none'), value=0.0), Dropdown(descrip…

In [53]:
storeDf.head()

,num,fileName,id,duration,success,finished,type,conversation
0,1,b6d828156a6f4f7e922ff4d708a8084b.json,A142ZRU284W9O,15,0.0,1.0,split,S: A142ZRU284W9O\n S: Pink is the target.\n L...
1,2,4fb0e0cb9b7e4330aa8f01c204c611a8.json,A35BY30TC8WCL4,25,0.0,1.0,split,"S: A35BY30TC8WCL4\n S: It's green, kind of.\n..."
2,4,4534f31223654992b295e86bba6b41fe.json,A22KRF782ELLB0,34,0.0,1.0,far,S: A22KRF782ELLB0\n S: pale green\n L: \n S: ...
3,5,051bb1abb9dc4518879590ccb7fbd826.json,A9EVF0SO2IUJ0,10,0.0,1.0,far,S: A9EVF0SO2IUJ0\n S: Blue\n L: \n S: Green\n
4,7,f1baeaa7d46b482d97761d1eb338030e.json,A2MKXI4KCRRI7Y,67,0.0,1.0,close,S: A2MKXI4KCRRI7Y\n S: lightest one\n L: The ...


In [65]:
a = storeDf.iloc[0,:]
a

num                                                             1
fileName                    b6d828156a6f4f7e922ff4d708a8084b.json
id                                                  A142ZRU284W9O
duration                                                       15
success                                                         0
finished                                                        1
type                                                        split
conversation     S: A142ZRU284W9O\n S: Pink is the target.\n L...
Name: 0, dtype: object